## The following modules will be used:
* BeautifulSoup - used to crawle the website
* pandas used to arrange data in tables

In [118]:
# Installation of the necessary libraries
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pymongo --quiet
!pip install urllib3 --quiet

In [119]:
import csv
import requests
import pandas as pd
import urllib
import time

from bs4 import BeautifulSoup
from pymongo import MongoClient
from IPython.display import display

In [120]:
%%bash
sudo apt-get update
sudo apt-get install -y mongodb-org > log
service mongodb start

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required
sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required
Failed to start mongodb.service: Unit mongodb.service not found.


CalledProcessError: Command 'b'sudo apt-get update\nsudo apt-get install -y mongodb-org > log\nservice mongodb start\n'' returned non-zero exit status 5.

In [ ]:
# Wait for the instance to start
time.sleep(5)

In [ ]:
%%bash
ps -ef | grep mongo

## Instantiate mongodb

In [30]:
mongoClient = MongoClient()
# Get the atlas mongodb connection string
# mongo_uri = "mongodb://vineethr238:" + urllib.parse.quote("vini8686", safe="") + "@cluster0.yqicrpo.mongodb.net/?retryWrites=true&w=majority"
# client = MongoClient(mongo_uri)
client = MongoClient()
client.list_database_names()
# db = client.stocknews
# store = db.stocks

# db = client.stocknews.stocks

['admin', 'config', 'local']

In [32]:
DATABASE_URI = "mongodb://localhost:27017"
DATABASE = "stocknews"
NEWS_COLLECTION = "stocks"
db = client[DATABASE]

# Check existence of collection
if "stocks" not in db.list_collection_names():
  db.create_collection(NEWS_COLLECTION)

## Test the Base Url

In [33]:
BASE_URL = 'https://finance.yahoo.com' 

## Utility Methods

In [44]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    response = requests.get(url)
    if not response.ok:
        print('Status code:', response.status_code)
        raise Exception('Failed to load page {}'.format(url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc


def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list


def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

def save_data_to_mongo(dataframe):
    """Takes a dataframe, converts it to dictionary and saves to mongo db"""
    #     data = dataframe.reset_index(drop=True)
    #     data_dictionary = data.to_dict(data)
    db.stocks.insert_many(dataframe.to_dict(orient="records"))

In [45]:


def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
    print('Requesting html page')
    doc = get_page(url)
    print('Extracting news tags')
    news_list = get_news_tags(doc)
    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]
    print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    news_df.to_csv(path, index=None)
    display(news_df.head())
    save_data_to_mongo(news_df)
    print("Successful pipeline execution")


if __name__ == "__main__" :
    
    YAHOO_NEWS_URL = BASE_URL+'/news/'
    news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

print("Processing Done")

Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV


,source,headline,url,content,image
0,Bloomberg,Asia Stocks Set to Open Lower on Hawkish Polic...,https://finance.yahoo.com/news/asia-stocks-set...,(Bloomberg) -- Asian stocks are poised to drop...,https://s.yimg.com/uu/api/res/1.2/tyrF3PUrZLoT...
1,Bloomberg,California Slashes Rooftop Solar Incentives in...,https://finance.yahoo.com/news/california-risk...,(Bloomberg) -- California will sharply reduce ...,https://s.yimg.com/uu/api/res/1.2/2HrpW53WntW7...
2,Reuters,Jana Partners tells Freshpet it plans to run f...,https://finance.yahoo.com/news/jana-partners-t...,The activist investment firm pushing Freshpet ...,https://s.yimg.com/uu/api/res/1.2/xLMB_HP1kGIc...
3,Reuters,Macau casinos' new contracts to sharpen focus ...,https://finance.yahoo.com/news/macau-casinos-c...,Macau's six incumbent casino firms are set to ...,https://s.yimg.com/uu/api/res/1.2/UqrJRRfqlwpZ...
4,Reuters,Macron wants European response to U.S. IRA to ...,https://finance.yahoo.com/news/macron-wants-eu...,French President Emmanuel Macron said on Thurs...,https://s.yimg.com/uu/api/res/1.2/Hy0uKJkwSftO...


Successful pipeline execution
Processing Done


In [48]:
# confirm data storage
db.stocks.count

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stocknews'), 'stocks.count')

In [50]:
for item in db.collection.find():
  print(item)